# XML parsing
https://docs.python.org/3/library/xml.etree.elementtree.html

In [ ]:
import xml.etree.ElementTree as ET

#tree = ET.parse('Results/Demand_CumulativeMaximumDemand_Arithmetic_Cmpo2100-2116.trx') # Importing XML file
tree = ET.parse('Results/Average_LastAveragePower_Arithmetic.trx') # Importing XML file

root = tree.getroot()

In [ ]:
print(root.tag)
print("********")
print(root.attrib)

In [ ]:
# Takojšnji podelementi XML datoteke
for child in root:
    print("Child tag \t ==> ", child.tag)
    print("Child attributes ==> ", child.attrib)
    print()

In [ ]:
# Start and finish time of the test
test_info = root.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Times")
print("Start time: ", test_info.attrib["start"])
print("Finish time: ", test_info.attrib["finish"])

In [ ]:
results = root.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Results")
print(results.tag)

for child in results:
    print("Child tag \t ==> ", child.tag)
    print("Child attributes ==> ", child.attrib)
    print()

In [ ]:
unit_test_results = results.findall("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}UnitTestResult") # findall checks only direct children

for test in unit_test_results:
    #print(test.attrib.keys())
    print(test.attrib["testName"])
    print(test.attrib["duration"])
    print(test.attrib["outcome"])
    if test.attrib["outcome"] == "Passed":
        print("No errors.")
    else:
        test_output = test.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Output")
        stdout = test_output.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}StdOut")
        stdout_parsed = parse_stdout(stdout.text)
        error_info = test_output.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}ErrorInfo")
        error_message = error_info.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Message")
        print(error_message.text)
    
    print()

# Excel report

https://realpython.com/openpyxl-excel-spreadsheets-python/

https://openpyxl.readthedocs.io/en/stable/styles.html

In [22]:
import xml.etree.ElementTree as ET
from openpyxl import Workbook
from openpyxl.styles import Font, Color, Alignment, Border, Side, colors, PatternFill
import glob
import random # to make random color
import re



def parse_stdout(data):
    print("PARSING STDOUT")
    # Add code to parse STDOUT
    
    


filename = "Results.xlsx"

workbook = Workbook() # creates new excell
sheet = workbook.active # selects the active sheet, which is the first one - Sheet 1

header = ["Sklop testov", 
            "Ime sklopa", 
            "Start set of tests", 
            "End set of tests", 
            "Sklop testov.1", 
            "Test name", 
            "Dolzina testa", 
            "Uspesnost",
            "Error info"]

for i, text in enumerate(header):
    cell = sheet.cell(row=1, column=i+1)
    cell.value = text
    #Styling
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")


'''
# Primer kako je sestavljen tests
tests = [
    ["Results/Average_Current", "0012e7cc"],
    ["Results/Average_LastAveragePower_Arithmetic", "001f536a"],
    ["Results/Average_LastAveragePower_Vector", "00f1b81d"],
    
]
'''

tests = []
trxFiles = glob.glob(f'Results/*.trx')
for file in trxFiles:
    #print(file)
    random_color = "00"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    tests.append([file, random_color]) # File_path, aRGB

for i in range(len(tests)):
    row = i+2
    sheet.cell(row=row, column=1).value = i
    fill = PatternFill(start_color=tests[i][1], # aRGB -> AARRGGBB
                   end_color=tests[i][1],
                   fill_type='solid')
    sheet.cell(row=row, column=1).fill = fill
    
    pattern = r"\\(.+)\."
    test_name = re.search(pattern, tests[i][0]).group(1)
    print("Test: ", test_name)
    sheet.cell(row=row, column=2).value = test_name
    sheet.cell(row=row, column=2).fill = fill
    
    tree = ET.parse(tests[i][0]) # Importing XML file
    root = tree.getroot()
    
    test_info = root.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Times")
    sheet.cell(row=row, column=3).value = test_info.attrib["start"]
    sheet.cell(row=row, column=4).value = test_info.attrib["finish"]
    
    results = root.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Results")
    unit_test_results = results.findall("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}UnitTestResult") # findall checks only direct children
    for test in unit_test_results:
        if sheet["F"][-1].value == None: # the first time, the value is None, because row has been createf for the first test
            row = sheet["F"][-1].row
        else: # the other times, this column is the one with most rows and so the last value is not None
            row = sheet["F"][-1].row + 1 
        
        sheet.cell(row=row, column=5).value = i
        sheet.cell(row=row, column=5).fill = fill
        
        sheet.cell(row=row, column=6).value = test.attrib["testName"]
        sheet.cell(row=row, column=7).value = test.attrib["duration"]
        
        sheet.cell(row=row, column=8).value = test.attrib["outcome"]
        if test.attrib["outcome"] == "Passed":
            outcome_fill = PatternFill(start_color='0000FF00', # aRGB -> AARRGGBB
                   end_color='0000FF00',
                   fill_type='solid')
            sheet.cell(row=row, column=8).fill = outcome_fill
            
            error_info_text = "No errors"
        else:
            outcome_fill = PatternFill(start_color='00FF0000',
                   end_color='00FF0000',
                   fill_type='solid')
            sheet.cell(row=row, column=8).fill = outcome_fill
            
            test_output = test.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Output")
            stdout = test_output.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}StdOut")
            stdout_parsed = parse_stdout(stdout.text)
            error_info = test_output.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}ErrorInfo")
            error_message = error_info.find("{http://microsoft.com/schemas/VisualStudio/TeamTest/2010}Message")
            
            error_info_text = error_message.text
            
        sheet.cell(row=row, column=9).value = error_info_text
            


workbook.save(filename=filename) # save

Test:  Average_Current
Test:  Average_LastAveragePowerFactor_Arithmetic_Lapf2100-2157
Test:  Average_LastAveragePowerFactor_Vector_Lapf1100-1157
Test:  Average_LastAveragePower_Arithmetic
PARSING STDOUT
Test:  Average_LastAveragePower_Vector
PARSING STDOUT
Test:  Average_MinimumPowerFactor_Arithmetic_Mipf2100-2407
Test:  Average_MinimumPowerFactor_Vector_Mipf1100-1407
Test:  Average_Power_Arithmetic_Avpo2600-2604
Test:  Average_Power_Vector_Avpo1600-1604
Test:  Average_Voltage
Test:  Demand_CumulativeMaximumDemand_Arithmetic_Cmpo2100-2116
PARSING STDOUT
PARSING STDOUT
Test:  Demand_CumulativeMaximumDemand_Arithmetic_Cmpo2200-2207
PARSING STDOUT
Test:  Demand_CumulativeMaximumDemand_Vector_Cmpo1100-1116
PARSING STDOUT
Test:  Demand_CumulativeMaximumDemand_Vector_Cmpo1200-1207
PARSING STDOUT
Test:  Demand_Demand_Arithmetic_Dere2600-2604
Test:  Demand_Demand_Vector_Dere1600-1604
Test:  Demand_MaximumDemand_Arithmetic_Made2100-2203
Test:  Demand_MaximumDemand_Arithmetic_Made2300-2503
PARSI

# Razlaga

In [1]:
import xml.etree.ElementTree as ET
import globlas


def xml_parser(folder_path):
    try:
        # Parse the XML content
        root = ET.fromstring(folder_path)

        # Iterate through all elements in the XML tree
        for element in root.iter("function"):
            name = element.get("name")
            if name == "Appliance_GoTo_TestMode__MAT":
                globals.go_to_test_mode = element.get("location")

            if name == "Appliance_RegisterInternalEvent__MAT":
                globals.go_to_standby_location = element.get("location")
                for element_mat in root.iter("enum"):
                    name = element_mat.get("name")
                    if name == "APPLIANCE_INTERNAL_EVENT__STANDBY_TIMER_ELAPSED":
                        globals.go_to_standby_value = int(element_mat.get("value"))

    except Exception as e:
        print(f"Error occurred while parsing the XML content: {e}")
        return None

In [12]:
import pandas as pd
from functools import lru_cache


@lru_cache(maxsize=5)
def load_xml(xml_file, xpath):
    return pd.read_xml(xml_file, xpath=xpath)


def extract_location_by_name(xml_file, function_name):
    df = load_xml("data/memoryLogCommands_02_01_09.xml", ".//function")
    location = df[df["name"] == function_name]["location"].values[0]
    return location


def extract_value_by_name(xml_file, enum_name):
    df = load_xml("data/memoryLogCommands_02_01_09.xml", ".//enum")
    value = df[df["name"] == enum_name]["value"].values[0]
    return value


location1 = extract_location_by_name(
    "data/memoryLogCommands_02_01_09.xml", "Appliance_GoTo_TestMode__MAT"
)
print(location1)
location2 = extract_location_by_name(
    "data/memoryLogCommands_02_01_09.xml", "Appliance_RegisterInternalEvent__MAT"
)
print(location2)

value = extract_value_by_name(
    "data/memoryLogCommands_02_01_09.xml",
    "APPLIANCE_INTERNAL_EVENT__STANDBY_TIMER_ELAPSED",
)
print(value)

0x08028c31
0x08028bc9


39
